In [4]:
# ![Python logo](https://www.python.org/static/community_logos/python-logo-master-v3-TM.png)

<img src="https://www.python.org/static/community_logos/python-logo-master-v3-TM.png"
    width='1000' 
    alt="alt_text" 
    style="display=block; margin:auto"
    />
    

***This is a brief guide to Python for users with a knowledge of another language and familiarity with recursion (for example, AP CSA students towards the end of the academic year). It uses minimax, a recursive algorithm which is well explained between 11:19 and 20:57 in [CS50 Minimax](https://www.youtube.com/watch?v=eey91kzfOZs&ab_channel=CS50).  If you're unfamiliar with minimax, it is highly recommended that you watch that part of the video first.
<br> The first part of the guide will focus on non-OOP concepts, and I'll introduce some basic OOP Python basics along with the OOP version of the game in the second part.  The OOP version is, in my opinion, cleaner and easier to write since the key aspects of the game's state can be shared among the methods of the class (ie, we don't have to think as hard about which functions need which arguments), but I will present it separately in order to make the exposition more gradual.***

<!-- # Table of contents
1. [Non-OOP Code](#nonoop)
2. [Some paragraph](#paragraph1)
    1. [Sub paragraph](#subparagraph1)
3. [Getting Help](#help)

## This is the introduction <a name="introduction"></a>
Some introduction text, formatted in heading 2 style

## Some paragraph <a name="paragraph1"></a>
The first paragraph text

### Sub paragraph <a name="subparagraph1"></a>
This is a sub paragraph, formatted in heading 3 style

## Another paragraph <a name="paragraph2"></a>
The second paragraph text -->

### PART 1: NON-OOP BASICS


[Lists](#lists)
[Modules](#modules)

_First, here's my non-OOP version of the code for the game of Tic-Tac-Toe.  The OOP version is presented later in this document_

___

## The non-OOP code for the game is below <a name="nonoop"></a>

```python
"""This program lets a person place the first piece on a tic-tac-toe board.  The computer uses the
minimax algorithm to ensure that the person never wins (a tie is possible).  The game is played
in the console and numbers for available position are displayed for the person playing.
 This feature requires the use of two auxiliary functions.  In addition, the best possible
 score and the best move are calculated in a single run of the minimax algorithm."""

import time  # will use this to introduce a pause between the person's and computer's choice

person_choices = []
computer_choices = []

gameboard = [['1', '|', '2', '|', '3'],
             ('-', '+', '-', '+', '-'),
             ['4', '|', '5', '|', '6'],
             ('-', '+', '-', '+', '-'),
             ['7', '|', '8', '|', '9']]


def main(person_choices, computer_choices, gameboard) -> None:
    user = "person"
    game_over = False
    depth = 0  # no moves yet, will track how many moves forward that the computer will evaluate the board
    print("Game board: ")
    print_gameboard(gameboard)

    while not game_over:  # Score starts with 0, so not score is true, once 10 or -10, not score will be false
        if user == "person":
            is_max = True
            make_move(gameboard, "person", depth, is_max)
            print_gameboard(gameboard)

            _, winner, game_over = check_winner(
                person_choices, computer_choices)

            if winner != "Tie" and winner != "":
                print(winner)
            elif game_over:
                print("Tie")
            time.sleep(.5)
            user = "computer"

        elif user == "computer":
            is_max = False
            make_move(gameboard, "computer", depth, is_max)
            print("Computer's move: ")
            print_gameboard(gameboard)

            _, winner, game_over = check_winner(
                person_choices, computer_choices)

            if winner != "Tie" and winner != "":
                print(winner)
            elif game_over:
                print("Tie")
            user = "person"


def print_gameboard(board) -> None:
    """Prints the board"""
    for i in range(0, len(board)):
        for j in range(0, len(board)):
            print(board[i][j], end=" ")
        print()


def make_move(board, user: str, depth: int, is_max: bool) -> None:
    """make_move function takes in the board, the user, depth, and is_max. 
    is_max is True if it's the person's term, False otherwise.  
    This function requires input from the user and
    definitively places the piece, hence it's not used in minimax"""
    while True:
        try:
            if user == "person":
                position = int(input("Please enter your placement 1 - 9: "))
                while (position in person_choices) or (position in computer_choices):
                    print(
                        "You either did not enter an integer between 1 and 9 or the position is taken")
                    position = int(
                        input("Please enter your placement 1 - 9: "))

            elif user == "computer":
                _, *bestMove = minimax(board, depth, is_max,
                                       person_choices, computer_choices)
                position = convert_to_pos(*bestMove)

        except ValueError as er:
            print("You must enter an integer", er)
        else:
            place_position(position, board, user)
            break


def place_position(position, board, user) -> None:
    """Places the user's piece at the specified position on the board"""
    symbol = "X" if user == "person" else "O"
    # fills out the first row of of the board
    position_helper(1, 4, 0, board, position, symbol)
    # fills out the second row of of the board
    position_helper(4, 7, 2, board, position, symbol)
    # fills out the third row of of the board
    position_helper(7, 10, 4, board, position, symbol)


def position_helper(a, b, c, board, position, symbol) -> None:
    """Used in the place_position function to avoid repetition"""
    for i in range(a, b):
        if position == i:
            board[c][2 * (i - a)] = symbol
            if symbol == 'X':
                person_choices.append(position)
            elif symbol == 'O':
                computer_choices.append(position)


def convert_to_pos(i, j) -> int:
    """Converts board coordinates to a position 1-9 on the board"""
    positionsDict = {(0, 0): 1, (0, 2): 2, (0, 4): 3, (2, 0): 4,
                     (2, 2): 5, (2, 4): 6, (4, 0): 7, (4, 2): 8, (4, 4): 9}
    return positionsDict[(i, j)]


def convert_pos_to_board(pos) -> tuple:
    """Converts a position 1-9 on the board to board coordinates """
    boardDict = {1: (0, 0), 2: (0, 2), 3: (0, 4), 4: (2, 0), 5: (
        2, 2), 6: (2, 4), 7: (4, 0), 8: (4, 2), 9: (4, 4)}
    return boardDict[pos]


def minimax(board, depth, is_max, person_choices, computer_choices) -> "tuple ('best_score', 'best_move_row', 'best_move_col')":
    """The minimax algorithm evaluates all future positions, assuming that the opponent
    plays optimally in the future.  It returns the score, row, and column of the best move"""
    user = "person" if is_max else "computer"
    score, *_ = check_winner(person_choices, computer_choices)
    if score == 10 or score == -10:
        return score, 1, 1  # come back to this and see if can fix
    if len(person_choices) + len(computer_choices) == 9:
        return 5, 0, 0

    if is_max:  # person's turn
        best_score, best_move_row, best_move_col = -1000, 1, 1
        for i in range(3):
            for j in range(3):
                if board[2 * i][2 * j] != "X" and board[2 * i][2 * j] != "O":
                    position = convert_to_pos(2 * i, 2 * j)
                    place_position(position, board, user)
                    score, *_ = minimax(
                        board, depth + 1, False, person_choices, computer_choices)
                    person_choices.pop()
                    m, n = convert_pos_to_board(position)
                    # undoing the move and rewriting the num on board
                    board[m][n] = position
                    if score > best_score:
                        best_score, best_move_row, best_move_col = score, m, n

    else:  # repetitive logic, but easier to see the algorithm at a glance this way
        best_score, best_move_row, best_move_col = 1000, -1, -1
        for i in range(3):
            for j in range(3):
                if board[2 * i][2 * j] != "X" and board[2 * i][2 * j] != "O":
                    position = convert_to_pos(2 * i, 2 * j)
                    place_position(position, board, user)
                    score, *_ = minimax(
                        board, depth + 1, True, person_choices, computer_choices)
                    computer_choices.pop()
                    m, n = convert_pos_to_board(position)
                    board[m][n] = position
                    if score < best_score:
                        best_score, best_move_row, best_move_col = score, m, n

    return best_score, best_move_row, best_move_col


def check_winner(person_choices, computer_choices) -> "tuple ('score', 'result', 'game_over')":
    """Given user, person_choices, and computer_choices, checks for the winner.
    Return the score, winner/tie/"", and True if the game is over/False if not"""
    all_winning = ((1, 2, 3), (4, 5, 6), (7, 8, 9), (1, 4, 7), (2, 5, 8), (3, 6, 9), (1, 5, 9),
                   (3, 5, 7))  # tuple of tuples of all winning positions

    for w in all_winning:
        if w[0] in person_choices and w[1] in person_choices and w[2] in person_choices:
            return 10, "Player wins", True

    for w in all_winning:
        if w[0] in computer_choices and w[1] in computer_choices and w[2] in computer_choices:
            return -10, "Computer wins", True

    if len(person_choices) + len(computer_choices) == 9:
        return 0, "Tie", True

    return 0, "", False


main(person_choices, computer_choices, gameboard)  
```


<a id = "modules"><span style='color:blue; font-size: 200%;font-family:Georgia'>Importing modules and packages</span></a>
<br><br>
Python has two main ways of importing packages or modules <br>
<br> __Method 1:__
```python
import module_or_package
```
__Method 2:__
```python 
from module_or_package import function_or_class_name
```
If you use Method 1, you'll need to reference the module/package followed by a dot and function/class name
#### ex: 
```python 
import time
time.sleep(.5) #sleep for half a second
```
If you use Method 2, you don't need to reference the module/package, but run a higher risk of namespace collisions
#### ex:
```python
from time import sleep
sleep(.5)
```
If you had your own function named 'sleep', you would overwrite the sleep function imported from time <br>
<br> __Notes__:
- When Python imports a module/package, it essentially pastes its code. It is a recommendeded good practice to place your import statements at the top of your program.

___


<span style='color:blue; font-size: 200%;font-family:Georgia'>Python Data Types</span>
<br><br>
The following link has a great summary [Python Data Types](https://medium.com/analytics-vidhya/data-types-in-python-506009234f89).
<br> There is an important distinction between _mutable_ and _immutable_ data type
- Lists, dictionaries, and sets are mutable, meaning that their elements can be reassigned to different values after creation
- Tuples, strings, Numbers, etc. are immutable
<br> For example, inside the gameboard list, there are lists (inside []) and tuples (inside ()).  Only the values inside lists can be changed.  If we try to change (_mutate_) a value inside one of the tuples, we will get an error.
```python

    gameboard = [['1', '|', '2', '|', '3'],
                ('-', '+', '-', '+', '-'),
                ['4', '|', '5', '|', '6'],
                ('-', '+', '-', '+', '-'),
                ['7', '|', '8', '|', '9']]
```
- Python has truthy and falsy values.  0, False, None, "", [], (), {}, set(), etc. are falsy, all other values are truthy.
- We will cover some key aspects of Python data structures below.

___

<a id="lists"><span style='color:blue; font-size: 200%;font-family:Georgia'> Lists in Python</span></a>
<br>
<br>Lists in Python are mutable, iterable, and indexable. These are somewhat similar to ArrayLists in Java and very similar to arrays in Javascript.  Unlike ArrayLists in Java, Python lists can contain elements of different data types.  In practice, however, lists are usually used to store elements of the same data type.

<br> Here are some basic operations on a list.
```python
>>> l = [0,2,4]
>>> l[1]=7
>>> l
[0, 7, 4]
>>> l.append(10)
>>> l
[0, 7, 4, 10]
>>> l.remove(4)
>>> l
[0, 7, 10]
>>> l.insert(0,100)
>>> l
[100, 0, 7, 10]
>>> l.sort()
>>> l
[0, 7, 10, 100]
>>> last = l.pop()
>>> last
100
>>> l
[0, 7, 10]
>>> l.extend([1,2,3])
>>> l
[0, 7, 10, 1, 2, 3]
```

In the following example, elements of the two-dimensional list _board_ are updated to symbol (symbol = "X" if user == "person" else "O").  The position is then appended to either the list of person or computer choices. 

```python 
def position_helper(a, b, c, board, position, symbol) -> None:
    """Used in the place_position function to avoid repetition"""
    for i in range(a, b):
        if position == i:
            board[c][2 * (i - a)] = symbol
            if symbol == 'X':
                person_choices.append(position)
            elif symbol == 'O':
                computer_choices.append(position)
```
___



<span style='color:blue; font-size: 200%;font-family:Georgia'>Tuples and Tuple Unpacking</span>
<br>
<br>Unlike lists, tuples are immutable.  They also often contain heterogeneous data types, as we will see in the examples of unpacked tuples below.  In addition, tuples are more performant than lists: as often there is a trade-off between performance and flexibility (the ability to mutate the list here).
<br> Tuples in Python are denoted mainly by commas and the parentheses are optional. They can be unpacked into individual variables.  For example, the function _check_winner_ returns a tuple containing three elements. We are not interested in assigning a value to the first element, and by convention _ is used for such variables, as in the following examples.  
```python
    _, winner, game_over = check_winner(
    person_choices, computer_choices)
    score, *_ = minimax(
                        board, depth + 1, False, person_choices, computer_choices)
```
In the second example, two variables were stored in _ since minimax returns a tuple of three elements.
We could, of course, also have stored them for future use as follows
```python
   score, *list_ = minimax(
                        board, depth + 1, False, person_choices, computer_choices)
```
Since minimax returns best_score, best_move_row, best_move_col, list_[0] would be best_move_row and list_[1] best_move_col. 
<br> _Please feel free to igore the Remark 1 on your first read.
<br>Sometimes a function may require a tuple argument.  For example, suppose we want to pass an (x,y) coordinate pair to function _describe_.  In this case, we would need to use the parentheses as in the following example. 
```python
    def describe(coord):
        print(f"This coordinate's x value is {coord[0]} and y value is {coord[1]}")

    describe((3,4))
    >>> This coordinate's x value is 3 and y value is 4
```
The _print_ function will be described below.  
<br> IMPORTANT: Notice how when * is used to the right of the equal sign, it packs the values into a tuple and when it is inside a function signature (or any place on the left of equal sign), it unpacks them.
```python
```

Remark 1: As a sidenote, list* is a list*. This is a built-in data type in Python, and an underscore can be used so that Python does not confuse it with the list constructor (more on OOP later, but if you're curious, see the sidenote below).

```python
>>> list=[1,2,3]
>>> list
[1, 2, 3]
>>> print(list)
[1, 2, 3]
>>> list('123')
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
TypeError: 'list' object is not callable
>>> type(list)
<class 'list'>
>>> list('Python')
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
TypeError: 'list' object is not callable
>>> del list
>>> list('Python')
['P', 'y', 't', 'h', 'o', 'n']

>>> list('123')
['1', '2', '3']

```
___


<span style='color:Blue; font-size: 200%;font-family:Georgia'> Dictionaries in Python </span>
<br><br>Dictionaries associate keys to values in Python.  Here's the general structure:
```python
dict_ = {key1:val1,key2:val2,...}
```
Since dict is a data type in Python, I'm using dict_.

<br> The keys must be hashable (immutable and comparable to other keys), which enables rapid retrieval of values.  In addition, the retrieval is convenient: if you had to retrieve an element from a list, you would either need to know its index or search for it in O(n) time.  If you're using a dictionary, you can retrieve a value present in dictionary without using an index or searching.  This can be done in approximately O(1) time.
<br> In the example below, a position number is passed as input and board coordinates are returned.  
```python
def convert_pos_to_board(pos) -> tuple:
    """Converts a position 1-9 on the board to board coordinates """
    boardDict = {1: (0, 0), 2: (0, 2), 3: (0, 4), 4: (2, 0), 5: (
        2, 2), 6: (2, 4), 7: (4, 0), 8: (4, 2), 9: (4, 4)}
    return boardDict[pos]
```
If an integer not between 1 and 9 were to be passed, we would get a KeyError.  Since we made sure that the integer is always between 1 and 9 in the make_move function, this will not happen in our code.  
<br>In general, though, if we are ever in a sitiation where we do not want our code to crash if a user is trying to pass in a key that's not in the dictionary, we could use the dictionary's _get_ method, replacing _boardDict[pos]_ with _boardDict.get(pos,0)_.  This method would return the value passed in as the second argument (0 in this case) if a key that's not in the dictionary is passed.

<br> By dictionaries contain key-value pairs.  By default, if we write _key in boardDict_, Python will loop through the keys.  If we want to loop through the values, we would use _val in boardDict.values()_
```python
boardDict = {1: (0, 0), 2: (0, 2), 3: (0, 4), 4: (2, 0), 5: (
        2, 2), 6: (2, 4), 7: (4, 0), 8: (4, 2), 9: (4, 4)}
evens_only = dict() #another way to instantiate a dictionary
for key in boardDict:
    if key%2 == 0:
        evens_only[key] = boardDict[key]
print(evens_only)
>>> {2: (0, 2), 4: (2, 0), 6: (2, 4), 8: (4, 2)}

```
If we want to perform some operation on the values of the dictionary, we can do this using  .values() method as in the following example.
```python
vals_add_to_even=[]
for val in boardDict.values():
    if (val[0]+val[1])%2 == 0:
        vals_add_to_even.append(val)
print(vals_add_to_even)
>>> [(0, 0), (0, 2), (0, 4), (2, 0), (2, 2), (2, 4), (4, 0), (4, 2), (4, 4)]

```
If we want to perform some operation on both keys and values of the dictionary, we can do this using .items() method as in the following example.

```python
evens_only = dict() 
vals_add_to_even=[]
for key,val in boardDict.items():
    if key%2 == 0:
        evens_only[key] = boardDict[key]
    if (val[0]+val[1])%2 == 0:
        vals_add_to_even.append(val)

print(evens_only)
print(vals_add_to_even)
>>> {2: (0, 2), 4: (2, 0), 6: (2, 4), 8: (4, 2)}
>>> [(0, 0), (0, 2), (0, 4), (2, 0), (2, 2), (2, 4), (4, 0), (4, 2), (4, 4)]
```

<br>Dictionaries in Python 3.7 and above preserve insertion order, they can be sorted, reversed, etc.  For more detail, please see https://docs.python.org/3/tutorial/datastructures.html#dictionaries .
___


<span style='color:Blue; font-size: 200%;font-family:Georgia'> The _print_ Function and f-strings</span>

The example above includes an f-string.  This is Python's way of doing string interpolation, which means evaluating the string at the specified values (3 for coord[0] and 4 for coord[1] above).  
<br> Python's _print_ function is very versatile and can take multiple inputs.
```python
print(1,2,3)
>>> 1 2 3
print("Hello","World")
>>> Hello World
```

The _sep_ keyword parameter can be configured to _separate_ the arguments by something other than " ".
```python
print(1,2,3, sep="***")
>>> 1***2***3
```

The _end_ keyword parameter can be configured to _end line_ with something other than "\n"

```python
print("Hello")
print("Python")
>>> Hello
>>> Python
```
```python
print("Hello",end="<--->")
print("Python")
```
>>> Hello<--->Python
___


<span style='color:Blue; font-size: 200%;font-family:Georgia'> Getting Help in Python</span>
<br><br>

To get help on a built-in function, one would use the help() function
```python
help(print)
>>>print(...)
    print(value, ..., sep=' ', end='\n', file=sys.stdout, flush=False)
    
    Prints the values to a stream, or to sys.stdout by default.
    Optional keyword arguments:
    file:  a file-like object (stream); defaults to the current sys.stdout.
    sep:   string inserted between values, default a space.
    end:   string appended after the last value, default a newline.
    flush: whether to forcibly flush the stream.
```
This function can also be run at the command line, which is also a good place to do quick experiments ;-)  To do this.
<br> python
<br> help(print)
>>>Help on built-in function print in module builtins:

print(...)
    print(value, ..., sep=' ', end='\n', file=sys.stdout, flush=False)...

Press q to exit help

Type exit() to exit Python.
___


<span style='color:Blue; font-size: 200%;font-family:Georgia'>Positional and Keyword Arguments</span>
<br><br>
There are more details here, but as a first-order approximation, the positional arguments are mandatory their corresponding parameters appear first in the signature.  The keyword arguments are optional. Their corresponding parameters follow the positional parameters and can be identified by the = sign.
<br>Let's examine the signature of the print function, namely 
```python 
print(value, ..., sep=' ', end='\n', file=sys.stdout, flush=False)
```
The function accepts multiple positional arguments as indicated by _value,..._, which is also commonly denoted by *args. The remaining arguments will be accepted as keyword arguments.  Please see the following example.

```python
>>> args=('hello','our','world')
>>> print(*args)
hello our world

>>> kwargs={'end':'???','sep':'!'}
>>> print(*args,**kwargs)
hello!our!world???>>> 

```
_kwargs_ is a dictionary, which we'll cover in more detail later.


In [ ]:
#### Indexing in Python

In [4]:
help(print)

Help on built-in function print in module builtins:

print(...)
    print(value, ..., sep=' ', end='\n', file=sys.stdout, flush=False)
    
    Prints the values to a stream, or to sys.stdout by default.
    Optional keyword arguments:
    file:  a file-like object (stream); defaults to the current sys.stdout.
    sep:   string inserted between values, default a space.
    end:   string appended after the last value, default a newline.
    flush: whether to forcibly flush the stream.



In [19]:
# from inspect import signature
# signature(describe)
# signature(print)

___

<span style='color:blue; font-size: 200%;font-family:Georgia'> Exception Handling in Python </span>
<br><br>
Let's examine the following function in detail (I removed the docstring for concision)
```python
def make_move(board, user: str, depth: int, is_max: bool) -> None:
    while True:
        try:
            if user == "person":
                position = int(input("Please enter your placement 1 - 9: "))
                while (position in person_choices) or \
                    (position in computer_choices):
                    print("You either did not enter an integer between 1 and 9 \ 
                    or the position is taken")
                    position = int(
                        input("Please enter your placement 1 - 9: "))

            elif user == "computer":
                _, *bestMove = minimax(board, depth, is_max,
                                       person_choices, computer_choices)
                position = convert_to_pos(*bestMove)

        except ValueError as er:
            print("You must enter an integer", er)
        else:
            place_position(position, board, user)
            break

````
If the user enters an input that cannot be converted to an integer (word 'five' instead of number 5, for example), then Python will throw a ValueError.  If we do not handle it, our program will crash.  In Python, exceptions can be handled with try-except statements.  We *try* to do something, and if it breaks, the code will go through the except statements from top to bottom (hence we should put the most specific ones towards the top, just as with if-elif-else statements).  The _else_ statement will run if no _except_ ran.  We can also have a _finally_ statement at the end to tie up the loose ends. The _finally_ statement will always run and is useful for many tasks (ex: displaying a final message, closing files, etc.)



<br> _Sidenote (can be skipped on first pass): Could we have handled the exceptions with classical if-else logic?  Probably yes, but this generally invovles more work.  Python's philosophy is EAFP: “it’s easier to ask for forgiveness than permission” language, not  LBYL: “look before you leap” language (such as Java). Python also has a notion of "duck typing", where we care more about the behavior of an object rather than it's type.  For example, we would find the sum over list or tuple elements in the same way, so we could write a sinle function that would work for both data types (see example below). Contrast this with a statically typed language such as Java, where objects (or primitive types) cannot change types after the initial declaration._   

```python
# Example of duck typing
def generic_sum(list_or_tuple):
    total = 0
    for elt in list_or_tuple:
        total += elt
    return total
print(generic_sum((1,2,3)))
print(generic_sum([1,2,3]))

```


In [5]:
def generic_sum(list_or_tuple):
    total = 0
    for elt in list_or_tuple:
        total += elt
    return total
print(generic_sum((1,2,3)))
print(generic_sum([1,2,3]))

6
6


___

<span style='color:blue; font-size: 200%;font-family:Georgia'> Default Values and Type Annotations in Python </span>
<br><br>
Python does not require us to specify the return and parameter types. This allows it to be more flexible, going along with the idea of duck typing.  However, as in example above, we can annotate some or all of the types for the parameters and the return value.  Python will ignore these annotations, but these can be helpful to another user or a tools used in conjunction with Python (linters, type checkers, IDEs, etc).
___

<span style='color:blue; font-size: 200%;font-family:Georgia'> List and Dictionary Comprehensions </span>
<br><br>Python is known for its powerful and flexible list comprehensions.  


<br>These are also more readable than map/apply/filter patterns which are available in Python, but would be the main resort in languages such as Javascript.  I'm not using list comprehensions in the code above, but I'll present some examples below.  _For the questions of removing all instances of a specified number from a list, how would you do it in Java?_

```python
#Squaring all elements in a list
l_1 = [1,2,3,4,5]
l_2 = [elt**2 for elt in l_1]
print(l_2)
#>>> [1, 4, 9, 16, 25]

l_2_evens = [elt**2 for elt in l_1 if elt**2 % 2 == 0]
print(l_2_evens)

#>>> [4, 16]

# A real-life use case: removing elements from a list
l = [1,2,3,2,2,2,4,5,6]
l.remove(2)
print(l)
#>>>[1, 3, 2, 2, 2, 4, 5, 6] # only removes the first 2!

l = [1,2,3,2,2,2,4,5,6]
l_proper = [elt for elt in l if elt != 2]
print(l_proper)

```
<br> Now let's refactor some of the code given in the Lists section using list and dictionary comprehensions.

```python
boardDict = {1: (0, 0), 2: (0, 2), 3: (0, 4), 4: (2, 0), 5: (
        2, 2), 6: (2, 4), 7: (4, 0), 8: (4, 2), 9: (4, 4)}
#BEFORE: 5 lines of code
evens_only = dict() #another way to instantiate a dictionary
for key in boardDict:
    if key%2 == 0:
        evens_only[key] = boardDict[key]
print(evens_only)
#>>> {2: (0, 2), 4: (2, 0), 6: (2, 4), 8: (4, 2)}
#AFTER: 2 lines of code
evens_only_compr = {key:boardDict[key] for key in boardDict if key%2 == 0}
print(evens_only_compr)
#>>> {2: (0, 2), 4: (2, 0), 6: (2, 4), 8: (4, 2)}

#BEFORE: 5 lines of code
vals_add_to_even=[]
for val in boardDict.values():
    if (val[0]+val[1])%2 == 0:
        vals_add_to_even.append(val)
print(vals_add_to_even)
#>>> [(0, 0), (0, 2), (0, 4), (2, 0), (2, 2), (2, 4), (4, 0), (4, 2), (4, 4)]

#AFTER: 2 lines of code
vals_add_to_even_compr = [val for val in boardDict.values() if (val[0]+val[1])%2 == 0]
print(vals_add_to_even_compr)
#>>> [(0, 0), (0, 2), (0, 4), (2, 0), (2, 2), (2, 4), (4, 0), (4, 2), (4, 4)]

```




### PART 2: OOP

<span style='color:blue; font-size: 200%;font-family:Georgia'> Classes and Objects </span>

Python is object-oriented but lacks the access modifiers (eg, private, protected, etc.) present in languages such as Java.  We will discuss solutions to this after we cover the basics.
<br> An OOP version of the game is below


In [ ]:
# """This program lets a person place the first piece on a Tic-Tac-Toe board.  The computer uses the
#     minimax algorithm to ensure that the person never wins (a tie is possible).  The game is played
#     in the console and numbers for available position are displayed for the person playing.
#     This feature requires the use of two auxiliary functions.  In addition, the best possible
#     score and the best move are calculated in a single run of the minimax algorithm."""

# import time  # will use this to introduce a pause between the person's and computer's choice


# class TicTacToeGame:
#     def __init__(self):
#         self.person_choices = []
#         self.computer_choices = []
#         self.gameboard = [['1', '|', '2', '|', '3'],
#                           ('-', '+', '-', '+', '-'),
#                           ['4', '|', '5', '|', '6'],
#                           ('-', '+', '-', '+', '-'),
#                           ['7', '|', '8', '|', '9']]

#     def run_game(self) -> None:
#         user = "person"
#         game_over = False
#         depth = 0  # no moves yet, will track how many moves forward that the computer will evaluate the board
#         print("Game board: ")
#         self.print_gameboard()

#         while not game_over:  # Score starts with 0, so not score is true, once 10 or -10, not score will be false
#             if user == "person":
#                 is_max = True
#                 self.make_move("person", depth, is_max)
#                 self.print_gameboard()

#                 _, winner, game_over = self.check_winner()

#                 if winner != "Tie" and winner != "":
#                     print(winner)
#                 elif game_over:
#                     print("Tie")
#                 time.sleep(.5)
#                 user = "computer"

#             elif user == "computer":
#                 is_max = False
#                 self.make_move("computer", depth, is_max)
#                 print("Computer's move: ")
#                 self.print_gameboard()

#                 _, winner, game_over = self.check_winner()

#                 if winner != "Tie" and winner != "":
#                     print(winner)
#                 elif game_over:
#                     print("Tie")
#                 user = "person"

#     def print_gameboard(self) -> None:
#         """Prints the board"""
#         for i in range(0, len(self.gameboard)):
#             for j in range(0, len(self.gameboard)):
#                 print(self.gameboard[i][j], end=" ")
#             print()

#     def make_move(self, user: str, depth: int, is_max: bool) -> None:
#         """make_move function takes in the board, the user, depth, and is_max. 
#         is_max is True if it's the person's term, False otherwise.  
#         This function requires input from the user and
#         definitively places the piece, hence it's not used in minimax"""
#         while True:
#             try:
#                 if user == "person":
#                     position = int(
#                         input("Please enter your placement 1 - 9: "))
#                     while (position in self.person_choices) or (position in self.computer_choices):
#                         print(
#                             "You either did not enter an integer between 1 and 9 or the position is taken")
#                         position = int(
#                             input("Please enter your placement 1 - 9: "))

#                 elif user == "computer":
#                     _, *bestMove = self.minimax(depth, is_max)
#                     position = self.convert_to_pos(*bestMove)

#             except ValueError as er:
#                 print("You must enter an integer", er)
#             else:
#                 self.place_position(position, user)
#                 break

#     def place_position(self, position, user) -> None:
#         """Places the user's piece at the specified position on the board"""
#         symbol = "X" if user == "person" else "O"
#         # fills out the first row of of the board
#         self.position_helper(1, 4, 0, position, symbol)
#         # fills out the second row of of the board
#         self.position_helper(4, 7, 2, position, symbol)
#         # fills out the third row of of the board
#         self.position_helper(7, 10, 4, position, symbol)

#     def position_helper(self, a, b, c, position, symbol) -> None:
#         """Used in the place_position function to avoid repetition"""
#         for i in range(a, b):
#             if position == i:
#                 self.gameboard[c][2 * (i - a)] = symbol
#                 if symbol == 'X':
#                     self.person_choices.append(position)
#                 elif symbol == 'O':
#                     self.computer_choices.append(position)

#     def convert_to_pos(self, i, j) -> int:
#         """Converts board coordinates to a position 1-9 on the board"""
#         positionsDict = {(0, 0): 1, (0, 2): 2, (0, 4): 3, (2, 0): 4,
#                          (2, 2): 5, (2, 4): 6, (4, 0): 7, (4, 2): 8, (4, 4): 9}
#         return positionsDict[(i, j)]

#     def convert_pos_to_board(self, pos) -> tuple:
#         """Converts a position 1-9 on the board to board coordinates """
#         boardDict = {1: (0, 0), 2: (0, 2), 3: (0, 4), 4: (2, 0), 5: (
#             2, 2), 6: (2, 4), 7: (4, 0), 8: (4, 2), 9: (4, 4)}
#         return boardDict[pos]

#     def minimax(self, depth, is_max) -> "tuple ('best_score', 'best_move_row', 'best_move_col')":
#         """The minimax algorithm evaluates all future positions, assuming that the opponent
#         plays optimally in the future.  It returns the score, row, and column of the best move"""
#         user = "person" if is_max else "computer"
#         score, *_ = self.check_winner()
#         if score == 10 or score == -10:
#             return score, 1, 1  # come back to this and see if can fix
#         if len(self.person_choices) + len(self.computer_choices) == 9:
#             return 5, 0, 0

#         if is_max:  # person's turn
#             best_score, best_move_row, best_move_col = -1000, 1, 1
#             for i in range(3):
#                 for j in range(3):
#                     if self.gameboard[2 * i][2 * j] != "X" and self.gameboard[2 * i][2 * j] != "O":
#                         position = self.convert_to_pos(2 * i, 2 * j)
#                         self.place_position(position, user)
#                         score, *_ = self.minimax(depth + 1, False)
#                         # The call above will build up a recursive stack and 'write' on the board + append
#                         # to person_choices and/or self.computer_choices.  Will need to undo these side-effects below
#                         # since all we want minimax to return is the best next move
#                         # Because place_position will call position_helper, which will append to person_choices
#                         self.person_choices.pop()
#                         m, n = self.convert_pos_to_board(position)
#                         # undoing the move and rewriting the num on board
#                         self.gameboard[m][n] = position
#                         if score > best_score:
#                             best_score, best_move_row, best_move_col = score, m, n

#         else:  # repetitive logic, but easier to see the algorithm at a glance this way
#             best_score, best_move_row, best_move_col = 1000, -1, -1
#             for i in range(3):
#                 for j in range(3):
#                     if self.gameboard[2 * i][2 * j] != "X" and self.gameboard[2 * i][2 * j] != "O":
#                         position = self.convert_to_pos(2 * i, 2 * j)
#                         self.place_position(position, user)
#                         score, *_ = self.minimax(depth + 1, True)
#                         self.computer_choices.pop()
#                         m, n = self.convert_pos_to_board(position)
#                         self.gameboard[m][n] = position
#                         if score < best_score:
#                             best_score, best_move_row, best_move_col = score, m, n

#         return best_score, best_move_row, best_move_col

#     def check_winner(self) -> "tuple ('score', 'result', 'game_over')":
#         """Given user, person_choices, and computer_choices, checks for the winner.
#         Return the score, winner/tie/"", and True if the game is over/False if not"""
#         all_winning = ((1, 2, 3), (4, 5, 6), (7, 8, 9), (1, 4, 7), (2, 5, 8), (3, 6, 9), (1, 5, 9),
#                        (3, 5, 7))  # tuple of tuples of all winning positions

#         for w in all_winning:
#             if w[0] in self.person_choices and w[1] in self.person_choices and w[2] in self.person_choices:
#                 return 10, "Player wins", True

#         for w in all_winning:
#             if w[0] in self.computer_choices and w[1] in self.computer_choices and w[2] in self.computer_choices:
#                 return -10, "Computer wins", True

#         if len(self.person_choices) + len(self.computer_choices) == 9:
#             return 0, "Tie", True

#         return 0, "", False


# game = TicTacToeGame()
# game.run_game()
